In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [2]:
import numpy as np
import phyloinfer as pinf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import defaultdict

In [3]:
from models import SBN

In [4]:
tree_dict_total = {}
tree_dict_map_total = defaultdict(float)
tree_names_total = []
tree_wts_total = []
n_samp_tree = 0
for i in range(1,11):
    tree_dict_rep, tree_name_rep, tree_wts_rep = pinf.result.mcmc_treeprob('../data/nchains_1/all_moves/rep_{}/DS1.trprobs'.format(i), 'nexus', taxon='keep')
    tree_wts_rep = np.round(np.array(tree_wts_rep)*750001)/750001
    
    for i, name in enumerate(tree_name_rep):
        tree_id = tree_dict_rep[name].get_topology_id()
        if tree_id not in tree_dict_map_total:
            n_samp_tree += 1
            tree_names_total.append('tree_{}'.format(n_samp_tree))
            tree_dict_total[tree_names_total[-1]] = tree_dict_rep[name]
            
        tree_dict_map_total[tree_id] += tree_wts_rep[i]

In [5]:
for key in tree_dict_map_total:
    tree_dict_map_total[key] /= 10
    
for name in tree_names_total:
    tree_wts_total.append(tree_dict_map_total[tree_dict_total[name].get_topology_id()])

In [4]:
from Bio import Phylo, File
from Bio.Phylo import NexusIO, Newick, NewickIO
from Bio.Nexus import Nexus

In [5]:
from cStringIO import StringIO

In [6]:
def node2clade(nxtree, node):
    subclades = [node2clade(nxtree, nxtree.node(n)) for n in node.succ]
    return Newick.Clade(
        branch_length=node.data.branchlength,
        name=node.data.taxon,
        clades=subclades,
        confidence=node.data.support,
        comment=node.data.comment)

In [12]:
mcmc_samp_tree_stats = Phylo.parse('../data/nchains_1/all_moves/rep_1/DS1.trprobs', 'nexus')

In [28]:
import pdb

In [110]:
for rep in range(2,11):
    tree_dict = defaultdict(list)
    # tree_name = []
    tree_count_dict = defaultdict(int)
#     skip = 250000

    with File.as_handle('../data/nchains_1/all_moves/rep_{}/DS1.t'.format(rep), 'rU') as fp:
        file_contents = fp.read()
        nex = Nexus.Nexus()
        nex.filename = getattr(fp, 'name', 'Unknown_nexus_file')

        file_contents = file_contents.strip()
        if file_contents.startswith('#NEXUS'):
            file_contents = file_contents[6:]
        commandlines = Nexus._get_command_lines(file_contents)
        for i, cl in enumerate(commandlines):
            try:
                if cl[:6].upper() == '#NEXUS':
                    commandlines[i] = cl[6:].strip()
            except IndexError:
                pass
        blocklines = []
        n_lines = 0
        n_samp_trees = 0
        finished = False
        while commandlines:        
            cl = commandlines.pop(0)
            n_lines += 1
            if cl.lower().startswith('begin'):
                title = cl.split()[1].lower()
            elif cl.lower().startswith('end'):
                finished = True
            elif n_lines>2 and skip:
                skip -= 1
                continue
            else:
                blocklines.append(cl)
            if n_lines > 2:
                n_samp_trees += 1
            if n_lines>2 and (n_samp_trees % 1000 == 0 or finished):
                nex._apply_block_structure(title, blocklines)
                block = nex.structured[-1]
                for line in block.commandlines:
                    getattr(nex, '_'+line.command)(line.options)


    #             pdb.set_trace()
                for nxtree in nex.trees:
                    newroot = node2clade(nxtree, nxtree.node(nxtree.root))
                    tree = Newick.Tree(root=newroot, rooted=nxtree.rooted, name=nxtree.name,
                              weight=nxtree.weight)
                    handle = StringIO()
                    Phylo.write(tree, handle, 'newick')
                    ete_tree = pinf.Tree(handle.getvalue().strip())
                    handle.close()
                    tree_id = ete_tree.get_topology_id()
                    if tree_id not in tree_dict:
                        tree_dict[tree_id].append(ete_tree)
                    tree_count_dict[tree_id] += 1

                print 'rep {}: read {} trees; {} unique trees'.format(rep, sum(tree_count_dict.values()), len(tree_count_dict))
                blocklines = []
                nex.trees = []

            if n_samp_trees in [5000, 10000, 20000, 40000, 80000, 160000, 320000, 640000]:
                np.save('../data/DS1/rep_{}/{}-samp-tree-dict'.format(rep, n_samp_trees), tree_dict)
                np.save('../data/DS1/rep_{}/{}-samp-tree-count-dict'.format(rep, n_samp_trees), tree_count_dict)

rep 2: read 1000 trees; 80 unique trees
rep 2: read 2000 trees; 103 unique trees
rep 2: read 3000 trees; 137 unique trees
rep 2: read 4000 trees; 147 unique trees
rep 2: read 5000 trees; 150 unique trees
rep 2: read 6000 trees; 153 unique trees
rep 2: read 7000 trees; 164 unique trees
rep 2: read 8000 trees; 196 unique trees
rep 2: read 9000 trees; 201 unique trees
rep 2: read 10000 trees; 207 unique trees
rep 2: read 11000 trees; 213 unique trees
rep 2: read 12000 trees; 223 unique trees
rep 2: read 13000 trees; 231 unique trees
rep 2: read 14000 trees; 233 unique trees
rep 2: read 15000 trees; 252 unique trees
rep 2: read 16000 trees; 271 unique trees
rep 2: read 17000 trees; 295 unique trees
rep 2: read 18000 trees; 310 unique trees
rep 2: read 19000 trees; 311 unique trees
rep 2: read 20000 trees; 315 unique trees
rep 2: read 21000 trees; 323 unique trees
rep 2: read 22000 trees; 329 unique trees
rep 2: read 23000 trees; 332 unique trees
rep 2: read 24000 trees; 338 unique trees
re

rep 2: read 195000 trees; 770 unique trees
rep 2: read 196000 trees; 771 unique trees
rep 2: read 197000 trees; 771 unique trees
rep 2: read 198000 trees; 771 unique trees
rep 2: read 199000 trees; 771 unique trees
rep 2: read 200000 trees; 772 unique trees
rep 2: read 201000 trees; 773 unique trees
rep 2: read 202000 trees; 774 unique trees
rep 2: read 203000 trees; 775 unique trees
rep 2: read 204000 trees; 779 unique trees
rep 2: read 205000 trees; 779 unique trees
rep 2: read 206000 trees; 780 unique trees
rep 2: read 207000 trees; 787 unique trees
rep 2: read 208000 trees; 787 unique trees
rep 2: read 209000 trees; 788 unique trees
rep 2: read 210000 trees; 788 unique trees
rep 2: read 211000 trees; 792 unique trees
rep 2: read 212000 trees; 793 unique trees
rep 2: read 213000 trees; 795 unique trees
rep 2: read 214000 trees; 797 unique trees
rep 2: read 215000 trees; 798 unique trees
rep 2: read 216000 trees; 799 unique trees
rep 2: read 217000 trees; 800 unique trees
rep 2: read

rep 2: read 386000 trees; 984 unique trees
rep 2: read 387000 trees; 984 unique trees
rep 2: read 388000 trees; 984 unique trees
rep 2: read 389000 trees; 985 unique trees
rep 2: read 390000 trees; 987 unique trees
rep 2: read 391000 trees; 989 unique trees
rep 2: read 392000 trees; 991 unique trees
rep 2: read 393000 trees; 991 unique trees
rep 2: read 394000 trees; 991 unique trees
rep 2: read 395000 trees; 992 unique trees
rep 2: read 396000 trees; 992 unique trees
rep 2: read 397000 trees; 992 unique trees
rep 2: read 398000 trees; 992 unique trees
rep 2: read 399000 trees; 992 unique trees
rep 2: read 400000 trees; 992 unique trees
rep 2: read 401000 trees; 994 unique trees
rep 2: read 402000 trees; 998 unique trees
rep 2: read 403000 trees; 998 unique trees
rep 2: read 404000 trees; 998 unique trees
rep 2: read 405000 trees; 999 unique trees
rep 2: read 406000 trees; 999 unique trees
rep 2: read 407000 trees; 1000 unique trees
rep 2: read 408000 trees; 1002 unique trees
rep 2: re

rep 2: read 573000 trees; 1115 unique trees
rep 2: read 574000 trees; 1117 unique trees
rep 2: read 575000 trees; 1117 unique trees
rep 2: read 576000 trees; 1118 unique trees
rep 2: read 577000 trees; 1119 unique trees
rep 2: read 578000 trees; 1119 unique trees
rep 2: read 579000 trees; 1119 unique trees
rep 2: read 580000 trees; 1122 unique trees
rep 2: read 581000 trees; 1125 unique trees
rep 2: read 582000 trees; 1126 unique trees
rep 2: read 583000 trees; 1130 unique trees
rep 2: read 584000 trees; 1130 unique trees
rep 2: read 585000 trees; 1134 unique trees
rep 2: read 586000 trees; 1135 unique trees
rep 2: read 587000 trees; 1136 unique trees
rep 2: read 588000 trees; 1139 unique trees
rep 2: read 589000 trees; 1139 unique trees
rep 2: read 590000 trees; 1139 unique trees
rep 2: read 591000 trees; 1140 unique trees
rep 2: read 592000 trees; 1140 unique trees
rep 2: read 593000 trees; 1141 unique trees
rep 2: read 594000 trees; 1141 unique trees
rep 2: read 595000 trees; 1143 u

rep 3: read 9000 trees; 210 unique trees
rep 3: read 10000 trees; 213 unique trees
rep 3: read 11000 trees; 221 unique trees
rep 3: read 12000 trees; 228 unique trees
rep 3: read 13000 trees; 234 unique trees
rep 3: read 14000 trees; 250 unique trees
rep 3: read 15000 trees; 255 unique trees
rep 3: read 16000 trees; 265 unique trees
rep 3: read 17000 trees; 272 unique trees
rep 3: read 18000 trees; 278 unique trees
rep 3: read 19000 trees; 287 unique trees
rep 3: read 20000 trees; 290 unique trees
rep 3: read 21000 trees; 295 unique trees
rep 3: read 22000 trees; 301 unique trees
rep 3: read 23000 trees; 307 unique trees
rep 3: read 24000 trees; 314 unique trees
rep 3: read 25000 trees; 317 unique trees
rep 3: read 26000 trees; 319 unique trees
rep 3: read 27000 trees; 322 unique trees
rep 3: read 28000 trees; 326 unique trees
rep 3: read 29000 trees; 334 unique trees
rep 3: read 30000 trees; 339 unique trees
rep 3: read 31000 trees; 347 unique trees
rep 3: read 32000 trees; 351 unique

...

rep 9: read 627000 trees; 1188 unique trees
rep 9: read 628000 trees; 1188 unique trees
rep 9: read 629000 trees; 1189 unique trees
rep 9: read 630000 trees; 1189 unique trees
rep 9: read 631000 trees; 1189 unique trees
rep 9: read 632000 trees; 1190 unique trees
rep 9: read 633000 trees; 1190 unique trees
rep 9: read 634000 trees; 1190 unique trees
rep 9: read 635000 trees; 1191 unique trees
rep 9: read 636000 trees; 1192 unique trees
rep 9: read 637000 trees; 1193 unique trees
rep 9: read 638000 trees; 1194 unique trees
rep 9: read 639000 trees; 1196 unique trees
rep 9: read 640000 trees; 1198 unique trees
rep 9: read 641000 trees; 1198 unique trees
rep 9: read 642000 trees; 1199 unique trees
rep 9: read 643000 trees; 1200 unique trees
rep 9: read 644000 trees; 1200 unique trees
rep 9: read 645000 trees; 1203 unique trees
rep 9: read 646000 trees; 1203 unique trees
rep 9: read 647000 trees; 1204 unique trees
rep 9: read 648000 trees; 1204 unique trees
rep 9: read 649000 trees; 1204 u

rep 10: read 64000 trees; 414 unique trees
rep 10: read 65000 trees; 427 unique trees
rep 10: read 66000 trees; 428 unique trees
rep 10: read 67000 trees; 429 unique trees
rep 10: read 68000 trees; 433 unique trees
rep 10: read 69000 trees; 435 unique trees
rep 10: read 70000 trees; 437 unique trees
rep 10: read 71000 trees; 439 unique trees
rep 10: read 72000 trees; 446 unique trees
rep 10: read 73000 trees; 446 unique trees
rep 10: read 74000 trees; 455 unique trees
rep 10: read 75000 trees; 467 unique trees
rep 10: read 76000 trees; 468 unique trees
rep 10: read 77000 trees; 468 unique trees
rep 10: read 78000 trees; 474 unique trees
rep 10: read 79000 trees; 480 unique trees
rep 10: read 80000 trees; 480 unique trees
rep 10: read 81000 trees; 484 unique trees
rep 10: read 82000 trees; 484 unique trees
rep 10: read 83000 trees; 486 unique trees
rep 10: read 84000 trees; 488 unique trees
rep 10: read 85000 trees; 492 unique trees
rep 10: read 86000 trees; 505 unique trees
rep 10: rea

rep 10: read 252000 trees; 828 unique trees
rep 10: read 253000 trees; 828 unique trees
rep 10: read 254000 trees; 830 unique trees
rep 10: read 255000 trees; 834 unique trees
rep 10: read 256000 trees; 834 unique trees
rep 10: read 257000 trees; 834 unique trees
rep 10: read 258000 trees; 834 unique trees
rep 10: read 259000 trees; 834 unique trees
rep 10: read 260000 trees; 835 unique trees
rep 10: read 261000 trees; 835 unique trees
rep 10: read 262000 trees; 835 unique trees
rep 10: read 263000 trees; 836 unique trees
rep 10: read 264000 trees; 837 unique trees
rep 10: read 265000 trees; 837 unique trees
rep 10: read 266000 trees; 837 unique trees
rep 10: read 267000 trees; 840 unique trees
rep 10: read 268000 trees; 841 unique trees
rep 10: read 269000 trees; 842 unique trees
rep 10: read 270000 trees; 844 unique trees
rep 10: read 271000 trees; 844 unique trees
rep 10: read 272000 trees; 847 unique trees
rep 10: read 273000 trees; 848 unique trees
rep 10: read 274000 trees; 849 u

rep 10: read 438000 trees; 1024 unique trees
rep 10: read 439000 trees; 1024 unique trees
rep 10: read 440000 trees; 1024 unique trees
rep 10: read 441000 trees; 1027 unique trees
rep 10: read 442000 trees; 1028 unique trees
rep 10: read 443000 trees; 1028 unique trees
rep 10: read 444000 trees; 1028 unique trees
rep 10: read 445000 trees; 1032 unique trees
rep 10: read 446000 trees; 1032 unique trees
rep 10: read 447000 trees; 1032 unique trees
rep 10: read 448000 trees; 1032 unique trees
rep 10: read 449000 trees; 1032 unique trees
rep 10: read 450000 trees; 1036 unique trees
rep 10: read 451000 trees; 1037 unique trees
rep 10: read 452000 trees; 1037 unique trees
rep 10: read 453000 trees; 1040 unique trees
rep 10: read 454000 trees; 1042 unique trees
rep 10: read 455000 trees; 1043 unique trees
rep 10: read 456000 trees; 1046 unique trees
rep 10: read 457000 trees; 1046 unique trees
rep 10: read 458000 trees; 1046 unique trees
rep 10: read 459000 trees; 1046 unique trees
rep 10: re

rep 10: read 621000 trees; 1156 unique trees
rep 10: read 622000 trees; 1156 unique trees
rep 10: read 623000 trees; 1156 unique trees
rep 10: read 624000 trees; 1159 unique trees
rep 10: read 625000 trees; 1159 unique trees
rep 10: read 626000 trees; 1159 unique trees
rep 10: read 627000 trees; 1159 unique trees
rep 10: read 628000 trees; 1160 unique trees
rep 10: read 629000 trees; 1160 unique trees
rep 10: read 630000 trees; 1161 unique trees
rep 10: read 631000 trees; 1161 unique trees
rep 10: read 632000 trees; 1161 unique trees
rep 10: read 633000 trees; 1161 unique trees
rep 10: read 634000 trees; 1161 unique trees
rep 10: read 635000 trees; 1163 unique trees
rep 10: read 636000 trees; 1163 unique trees
rep 10: read 637000 trees; 1163 unique trees
rep 10: read 638000 trees; 1163 unique trees
rep 10: read 639000 trees; 1163 unique trees
rep 10: read 640000 trees; 1163 unique trees
rep 10: read 641000 trees; 1164 unique trees
rep 10: read 642000 trees; 1165 unique trees
rep 10: re

In [69]:
sorted([(key, value) for key, value in tree_count_dict.iteritems()], key=lambda x:x[1], reverse=True)

[('86c836a5a03165112825a17412846399', 210761),
 ('0337a117bb568a1797b28695a52b0294', 149971),
 ('e1edc33927b893e2196e3658ec6215eb', 47316),
 ('1435e0924ccf935736718e646cc80e99', 46726),
 ('f1af2730cb2738bf87a6588734773e0f', 36370),
 ('0840d8703dda6ccd2364491f926d6a3a', 32950),
 ('ff2a89e37d6149d6bafdac84b043e2ba', 25352),
 ('a44652443bb48ebfc5e2732d3f190e58', 14502),
 ('0420e289d6ff4ee072c77af5f6a62405', 13089),
 ('4b5da39a83d867cf417627e521c107e2', 13081),
 ('8e93b349d740f8d63dd551c9d0d193a2', 12742),
 ('4befb206414eeb4382c0e71368417119', 8881),
 ('cfe3f7f3a3b26697329608206662c956', 8376),
 ('d6c35b29c55acc7f5726f65c123e76da', 8270),
 ('c7fb13797e4c10cc99e20db9b1d3b8db', 7870),
 ('636d7a920f7d9c9bb0d4386f27722546', 7614),
 ('fdb93104936abf47fc43375dfaa37934', 5661),
 ('c0576710b2c4a1ae7c4e756d8a308055', 5211),
 ('da1fae3650a0880d5d2384a8b645a4b7', 4989),
 ('7773ce85c60a23410e0f099a2e65f6f8', 3588),
 ('465531a8c2e159ce4cfa19353a026858', 3571),
 ('2297d4917d90267b72512e626c76551f', 3473

In [106]:
tree_dict_test = np.load('../data/DS1/rep_1/640000-samp-tree-dict.npy').item()

In [107]:
tree_count_dict_test = np.load('../data/DS1/rep_1/640000-samp-tree-count-dict.npy').item()

In [108]:
len(tree_count_dict_test)

1143

In [109]:
sorted([(key, value) for key, value in tree_count_dict_test.iteritems()], key=lambda x:x[1], reverse=True)

[('86c836a5a03165112825a17412846399', 178620),
 ('0337a117bb568a1797b28695a52b0294', 127098),
 ('1435e0924ccf935736718e646cc80e99', 41132),
 ('e1edc33927b893e2196e3658ec6215eb', 40067),
 ('f1af2730cb2738bf87a6588734773e0f', 30754),
 ('0840d8703dda6ccd2364491f926d6a3a', 27877),
 ('ff2a89e37d6149d6bafdac84b043e2ba', 21394),
 ('a44652443bb48ebfc5e2732d3f190e58', 12711),
 ('4b5da39a83d867cf417627e521c107e2', 11524),
 ('0420e289d6ff4ee072c77af5f6a62405', 11496),
 ('8e93b349d740f8d63dd551c9d0d193a2', 10845),
 ('4befb206414eeb4382c0e71368417119', 7581),
 ('d6c35b29c55acc7f5726f65c123e76da', 7295),
 ('cfe3f7f3a3b26697329608206662c956', 7085),
 ('c7fb13797e4c10cc99e20db9b1d3b8db', 6937),
 ('636d7a920f7d9c9bb0d4386f27722546', 6463),
 ('fdb93104936abf47fc43375dfaa37934', 4992),
 ('c0576710b2c4a1ae7c4e756d8a308055', 4395),
 ('da1fae3650a0880d5d2384a8b645a4b7', 4390),
 ('465531a8c2e159ce4cfa19353a026858', 3114),
 ('2297d4917d90267b72512e626c76551f', 3001),
 ('7773ce85c60a23410e0f099a2e65f6f8', 2995

In [89]:
sorted([(key, value) for key, value in tree_count_dict_test.iteritems()], key=lambda x:x[1], reverse=True)

[('86c836a5a03165112825a17412846399', 2853),
 ('0337a117bb568a1797b28695a52b0294', 2009),
 ('e1edc33927b893e2196e3658ec6215eb', 612),
 ('1435e0924ccf935736718e646cc80e99', 580),
 ('f1af2730cb2738bf87a6588734773e0f', 501),
 ('0840d8703dda6ccd2364491f926d6a3a', 474),
 ('ff2a89e37d6149d6bafdac84b043e2ba', 324),
 ('a44652443bb48ebfc5e2732d3f190e58', 194),
 ('8e93b349d740f8d63dd551c9d0d193a2', 177),
 ('0420e289d6ff4ee072c77af5f6a62405', 176),
 ('4b5da39a83d867cf417627e521c107e2', 172),
 ('4befb206414eeb4382c0e71368417119', 136),
 ('d6c35b29c55acc7f5726f65c123e76da', 109),
 ('636d7a920f7d9c9bb0d4386f27722546', 102),
 ('c7fb13797e4c10cc99e20db9b1d3b8db', 98),
 ('cfe3f7f3a3b26697329608206662c956', 98),
 ('fdb93104936abf47fc43375dfaa37934', 69),
 ('c0576710b2c4a1ae7c4e756d8a308055', 67),
 ('da1fae3650a0880d5d2384a8b645a4b7', 58),
 ('7773ce85c60a23410e0f099a2e65f6f8', 52),
 ('465531a8c2e159ce4cfa19353a026858', 47),
 ('9b1691dc3057dc083bdf694445ca6d4f', 44),
 ('331259e4f551dde83247f7f2f1284259', 

In [8]:
import pdb

In [36]:
tree_dict = defaultdict(list)
# tree_name = []
tree_wts_dict = defaultdict(int)
n_samp_trees_total = 0

for rep in range(1,11):

#     skip = 250000
    skip = 0

    with File.as_handle('../data/raw_data_DS1-11/DS6/rep_{}/DS6.trprobs'.format(rep), 'rU') as fp:
        file_contents = fp.read()
        nex = Nexus.Nexus()
        nex.filename = getattr(fp, 'name', 'Unknown_nexus_file')

        file_contents = file_contents.strip()
        if file_contents.startswith('#NEXUS'):
            file_contents = file_contents[6:]
        commandlines = Nexus._get_command_lines(file_contents)
        for i, cl in enumerate(commandlines):
            try:
                if cl[:6].upper() == '#NEXUS':
                    commandlines[i] = cl[6:].strip()
            except IndexError:
                pass
        blocklines = []
        n_lines = 0
        n_samp_trees = 0
        finished = False
        while commandlines:        
            cl = commandlines.pop(0)
            n_lines += 1
            if cl.lower().startswith('begin'):
                title = cl.split()[1].lower()
            elif cl.lower().startswith('end'):
                finished = True
            elif n_lines>2 and skip:
                skip -= 1
                continue
            else:
                blocklines.append(cl)
            if n_lines > 2:
                n_samp_trees += 1
            if n_lines>2 and (n_samp_trees % 1000 == 0 or finished):
                nex._apply_block_structure(title, blocklines)
                block = nex.structured[-1]
                for line in block.commandlines:
                    getattr(nex, '_'+line.command)(line.options)

                for nxtree in nex.trees:
                    newroot = node2clade(nxtree, nxtree.node(nxtree.root))
                    tree = Newick.Tree(root=newroot, rooted=nxtree.rooted, name=nxtree.name,
                              weight=nxtree.weight)
                    handle = StringIO()
                    Phylo.write(tree, handle, 'newick', plain=True)
                    newick_tree = handle.getvalue().strip()
                    ete_tree = pinf.Tree(newick_tree)
                    handle.close()
                    tree_id = ete_tree.get_topology_id()
                    if tree_id not in tree_dict:
                        tree_dict[tree_id].append(newick_tree)
                    tree_wts_dict[tree_id] += np.round(nxtree.weight*750001)

                print 'rep {}: {} trees scanned; {} unique trees'.format(rep, sum(tree_wts_dict.values()), len(tree_wts_dict))
                blocklines = []
                nex.trees = []

for key in tree_wts_dict:
    tree_wts_dict[key] /= 10*750001

#             if n_samp_trees in [5000, 10000, 20000, 40000, 80000, 160000, 320000, 640000]:
#                 np.save('../data/DS1/rep_{}/{}-samp-tree-dict'.format(rep, n_samp_trees), tree_dict)
#                 np.save('../data/DS1/rep_{}/{}-samp-tree-count-dict'.format(rep, n_samp_trees), tree_count_dict)

rep 1: 143079.0 trees completed; 1000 unique trees
rep 1: 212571.0 trees completed; 2000 unique trees
rep 1: 258641.0 trees completed; 3000 unique trees
rep 1: 290979.0 trees completed; 4000 unique trees
rep 1: 315556.0 trees completed; 5000 unique trees
rep 1: 335634.0 trees completed; 6000 unique trees
rep 1: 352710.0 trees completed; 7000 unique trees
rep 1: 367679.0 trees completed; 8000 unique trees
rep 1: 381119.0 trees completed; 9000 unique trees
rep 1: 393288.0 trees completed; 10000 unique trees
rep 1: 404410.0 trees completed; 11000 unique trees
rep 1: 414641.0 trees completed; 12000 unique trees
rep 1: 424238.0 trees completed; 13000 unique trees
rep 1: 433238.0 trees completed; 14000 unique trees
rep 1: 441551.0 trees completed; 15000 unique trees
rep 1: 449551.0 trees completed; 16000 unique trees
rep 1: 456946.0 trees completed; 17000 unique trees
rep 1: 463946.0 trees completed; 18000 unique trees
rep 1: 470946.0 trees completed; 19000 unique trees
rep 1: 476965.0 trees

rep 1: 713067.0 trees completed; 158000 unique trees
rep 1: 714067.0 trees completed; 159000 unique trees
rep 1: 715067.0 trees completed; 160000 unique trees
rep 1: 716067.0 trees completed; 161000 unique trees
rep 1: 717067.0 trees completed; 162000 unique trees
rep 1: 718067.0 trees completed; 163000 unique trees
rep 1: 719067.0 trees completed; 164000 unique trees
rep 1: 720067.0 trees completed; 165000 unique trees
rep 1: 721067.0 trees completed; 166000 unique trees
rep 1: 722067.0 trees completed; 167000 unique trees
rep 1: 723067.0 trees completed; 168000 unique trees
rep 1: 724067.0 trees completed; 169000 unique trees
rep 1: 725067.0 trees completed; 170000 unique trees
rep 1: 726067.0 trees completed; 171000 unique trees
rep 1: 727067.0 trees completed; 172000 unique trees
rep 1: 728067.0 trees completed; 173000 unique trees
rep 1: 729067.0 trees completed; 174000 unique trees
rep 1: 730067.0 trees completed; 175000 unique trees
rep 1: 731067.0 trees completed; 176000 unique

rep 2: 1421049.0 trees completed; 243761 unique trees
rep 2: 1422049.0 trees completed; 244586 unique trees
rep 2: 1423049.0 trees completed; 245342 unique trees
rep 2: 1424049.0 trees completed; 246151 unique trees
rep 2: 1425049.0 trees completed; 246926 unique trees
rep 2: 1426049.0 trees completed; 247693 unique trees
rep 2: 1427049.0 trees completed; 248511 unique trees
rep 2: 1428049.0 trees completed; 249254 unique trees
rep 2: 1429049.0 trees completed; 249953 unique trees
rep 2: 1430049.0 trees completed; 250651 unique trees
rep 2: 1431049.0 trees completed; 251448 unique trees
rep 2: 1432049.0 trees completed; 252165 unique trees
rep 2: 1433049.0 trees completed; 252990 unique trees
rep 2: 1434049.0 trees completed; 253791 unique trees
rep 2: 1435049.0 trees completed; 254529 unique trees
rep 2: 1436049.0 trees completed; 255296 unique trees
rep 2: 1437049.0 trees completed; 256076 unique trees
rep 2: 1438049.0 trees completed; 256780 unique trees
rep 2: 1439049.0 trees compl

rep 3: 2127898.0 trees completed; 311578 unique trees
rep 3: 2128898.0 trees completed; 312144 unique trees
rep 3: 2129898.0 trees completed; 312713 unique trees
rep 3: 2130898.0 trees completed; 313296 unique trees
rep 3: 2131898.0 trees completed; 313820 unique trees
rep 3: 2132898.0 trees completed; 314387 unique trees
rep 3: 2133898.0 trees completed; 314964 unique trees
rep 3: 2134898.0 trees completed; 315553 unique trees
rep 3: 2135898.0 trees completed; 316177 unique trees
rep 3: 2136898.0 trees completed; 316740 unique trees
rep 3: 2137898.0 trees completed; 317314 unique trees
rep 3: 2138898.0 trees completed; 317896 unique trees
rep 3: 2139898.0 trees completed; 318460 unique trees
rep 3: 2140898.0 trees completed; 319089 unique trees
rep 3: 2141898.0 trees completed; 319705 unique trees
rep 3: 2142898.0 trees completed; 320287 unique trees
rep 3: 2143898.0 trees completed; 320908 unique trees
rep 3: 2144898.0 trees completed; 321665 unique trees
rep 3: 2145898.0 trees compl

rep 4: 2772648.0 trees completed; 390318 unique trees
rep 4: 2776648.0 trees completed; 390325 unique trees
rep 4: 2780648.0 trees completed; 390336 unique trees
rep 4: 2784648.0 trees completed; 390342 unique trees
rep 4: 2788648.0 trees completed; 390355 unique trees
rep 4: 2792530.0 trees completed; 390373 unique trees
rep 4: 2795530.0 trees completed; 390419 unique trees
rep 4: 2798530.0 trees completed; 390452 unique trees
rep 4: 2801530.0 trees completed; 390488 unique trees
rep 4: 2804530.0 trees completed; 390529 unique trees
rep 4: 2807530.0 trees completed; 390569 unique trees
rep 4: 2810530.0 trees completed; 390604 unique trees
rep 4: 2813530.0 trees completed; 390648 unique trees
rep 4: 2816530.0 trees completed; 390695 unique trees
rep 4: 2819530.0 trees completed; 390728 unique trees
rep 4: 2822530.0 trees completed; 390772 unique trees
rep 4: 2825530.0 trees completed; 390816 unique trees
rep 4: 2827583.0 trees completed; 390960 unique trees
rep 4: 2829583.0 trees compl

...

rep 7: 5087013.0 trees completed; 596999 unique trees
rep 7: 5089013.0 trees completed; 597082 unique trees
rep 7: 5091013.0 trees completed; 597159 unique trees
rep 7: 5093013.0 trees completed; 597230 unique trees
rep 7: 5095013.0 trees completed; 597285 unique trees
rep 7: 5097013.0 trees completed; 597341 unique trees
rep 7: 5099013.0 trees completed; 597420 unique trees
rep 7: 5101013.0 trees completed; 597503 unique trees
rep 7: 5103013.0 trees completed; 597568 unique trees
rep 7: 5105013.0 trees completed; 597641 unique trees
rep 7: 5107013.0 trees completed; 597731 unique trees
rep 7: 5109013.0 trees completed; 597819 unique trees
rep 7: 5111013.0 trees completed; 597891 unique trees
rep 7: 5113013.0 trees completed; 597971 unique trees
rep 7: 5115013.0 trees completed; 598054 unique trees
rep 7: 5117013.0 trees completed; 598133 unique trees
rep 7: 5119013.0 trees completed; 598209 unique trees
rep 7: 5121013.0 trees completed; 598273 unique trees
rep 7: 5123013.0 trees compl

rep 8: 5603252.0 trees completed; 654741 unique trees
rep 8: 5618138.0 trees completed; 654741 unique trees
rep 8: 5631544.0 trees completed; 654741 unique trees
rep 8: 5643732.0 trees completed; 654741 unique trees
rep 8: 5654835.0 trees completed; 654741 unique trees
rep 8: 5665062.0 trees completed; 654741 unique trees
rep 8: 5674649.0 trees completed; 654741 unique trees
rep 8: 5683649.0 trees completed; 654741 unique trees
rep 8: 5691916.0 trees completed; 654741 unique trees
rep 8: 5699916.0 trees completed; 654741 unique trees
rep 8: 5707232.0 trees completed; 654741 unique trees
rep 8: 5714232.0 trees completed; 654741 unique trees
rep 8: 5721181.0 trees completed; 654741 unique trees
rep 8: 5727181.0 trees completed; 654741 unique trees
rep 8: 5733181.0 trees completed; 654741 unique trees
rep 8: 5739181.0 trees completed; 654741 unique trees
rep 8: 5744582.0 trees completed; 654741 unique trees
rep 8: 5749582.0 trees completed; 654741 unique trees
rep 8: 5754582.0 trees compl

rep 8: 5964442.0 trees completed; 695441 unique trees
rep 8: 5965442.0 trees completed; 695950 unique trees
rep 8: 5966442.0 trees completed; 696345 unique trees
rep 8: 5967442.0 trees completed; 696770 unique trees
rep 8: 5968442.0 trees completed; 697218 unique trees
rep 8: 5969442.0 trees completed; 697654 unique trees
rep 8: 5970442.0 trees completed; 698071 unique trees
rep 8: 5971442.0 trees completed; 698444 unique trees
rep 8: 5972442.0 trees completed; 698761 unique trees
rep 8: 5973442.0 trees completed; 699136 unique trees
rep 8: 5974442.0 trees completed; 699459 unique trees
rep 8: 5975442.0 trees completed; 699871 unique trees
rep 8: 5976442.0 trees completed; 700252 unique trees
rep 8: 5977442.0 trees completed; 700562 unique trees
rep 8: 5978442.0 trees completed; 700905 unique trees
rep 8: 5979442.0 trees completed; 701265 unique trees
rep 8: 5980442.0 trees completed; 701690 unique trees
rep 8: 5981442.0 trees completed; 702163 unique trees
rep 8: 5982442.0 trees compl

rep 9: 6671288.0 trees completed; 730560 unique trees
rep 9: 6672288.0 trees completed; 731007 unique trees
rep 9: 6673288.0 trees completed; 731445 unique trees
rep 9: 6674288.0 trees completed; 731833 unique trees
rep 9: 6675288.0 trees completed; 732238 unique trees
rep 9: 6676288.0 trees completed; 732598 unique trees
rep 9: 6677288.0 trees completed; 732884 unique trees
rep 9: 6678288.0 trees completed; 733162 unique trees
rep 9: 6679288.0 trees completed; 733480 unique trees
rep 9: 6680288.0 trees completed; 733771 unique trees
rep 9: 6681288.0 trees completed; 734080 unique trees
rep 9: 6682288.0 trees completed; 734422 unique trees
rep 9: 6683288.0 trees completed; 734740 unique trees
rep 9: 6684288.0 trees completed; 735019 unique trees
rep 9: 6685288.0 trees completed; 735393 unique trees
rep 9: 6686288.0 trees completed; 735738 unique trees
rep 9: 6687288.0 trees completed; 736214 unique trees
rep 9: 6688288.0 trees completed; 736706 unique trees
rep 9: 6689288.0 trees compl

rep 10: 7376909.0 trees completed; 762725 unique trees
rep 10: 7377909.0 trees completed; 763067 unique trees
rep 10: 7378909.0 trees completed; 763419 unique trees
rep 10: 7379909.0 trees completed; 763793 unique trees
rep 10: 7380909.0 trees completed; 764195 unique trees
rep 10: 7381909.0 trees completed; 764619 unique trees
rep 10: 7382909.0 trees completed; 764987 unique trees
rep 10: 7383909.0 trees completed; 765396 unique trees
rep 10: 7384909.0 trees completed; 765883 unique trees
rep 10: 7385909.0 trees completed; 766475 unique trees
rep 10: 7386909.0 trees completed; 766894 unique trees
rep 10: 7387909.0 trees completed; 767458 unique trees
rep 10: 7388909.0 trees completed; 768016 unique trees
rep 10: 7389909.0 trees completed; 768534 unique trees
rep 10: 7390909.0 trees completed; 769074 unique trees
rep 10: 7391909.0 trees completed; 769417 unique trees
rep 10: 7392909.0 trees completed; 769780 unique trees
rep 10: 7393909.0 trees completed; 770098 unique trees
rep 10: 73

In [37]:
emp_tree_freq = {newick_tree[0]:tree_wts_dict[tree_id] for tree_id, newick_tree in tree_dict.iteritems()}

In [38]:
len(emp_tree_freq)

809765

In [39]:
sum(emp_tree_freq.values())

0.99999999999242872

In [34]:
type(tree)

ete3.coretype.tree.TreeNode

In [40]:
tree = pinf.Tree(emp_tree_freq.keys()[0])

In [41]:
type(tree)

ete3.coretype.tree.TreeNode

In [43]:
np.save('../data/data_DS1-11/DS6/emp_tree_freq', emp_tree_freq)